# 10.3.3 Scrape Mars Data : The news

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd


In [88]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\tsanzo\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


In [3]:
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)


True

In [4]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')


In [5]:
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title


"HiRISE Views NASA's InSight and Curiosity on Mars"

In [6]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'New images taken from space offer the clearest orbital glimpse yet of InSight as well as a view of Curiosity rolling along.'

# 10.3.4 Scrap Mars Data : Featured Images

## Featured Images

In [7]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [15]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [16]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [18]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [19]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

# 10.3.5 Scrape Mars Data : Mars Facts

In [33]:
#Use read_html[0] to extract first table in website


df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

In [114]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [117]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.

# Retrieving product items that contain the links to full res image URL's and titles
html = browser.html
hemi_soup = soup(html, 'html.parser')
product_elems = hemi_soup.findAll('a', class_ = 'product-item')[:-1:2] 
#there are 9 total 'product-item' classes. the last is one we aren't interested in and
#the first 8 are the 4 we want but duplicated which is why a step of 2 is used


#loop through each element in product elem list 
for elem in product_elems:

    #visit each link using the href tag(link to each hemisphere article)
    browser.visit(url+elem.get('href'))

    #save and soup the html
    html = browser.html
    hemi_soup = soup(html, 'html.parser')

    #methods to retrieve each title and image link by tags
    title = hemi_soup.find('h2').text
    rel_image_link = hemi_soup.findAll('a', target = '_blank')[2].get('href')

    #append dictionary of title and link to image url list
    hemisphere_image_urls.append({'img_url': url+rel_image_link, 'title': title})

#return to home page 
browser.visit(url)

In [119]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [120]:
# 5. Quit the browser
browser.quit()